In [1]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import cv2
import numpy as np
from sklearn.utils import shuffle
from sklearn import model_selection
from matplotlib import pyplot as plt

import tensorflow as tf
#from keras_squeezenet import SqueezeNet
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import  CategoricalCrossentropy
from tensorflow.keras import utils
from tensorflow.keras.layers import Activation, Dropout, Convolution2D, GlobalAveragePooling2D

In [3]:
main_directory = 'dataset'

In [4]:
class_map = {
    "rock": 0,
    "paper": 1,
    "scissors": 2,
    "lizard": 3,
    "spock": 4,
    "nothing": 5
}

In [6]:
dataset = []
for directory in os.listdir(main_directory):
    path = os.path.join(main_directory, directory)
    if not os.path.isdir(path):
        continue
    for item in os.listdir(path):
        image = cv2.imread(os.path.join(path, item))
        #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        #image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        image = cv2.resize(image, (300, 300))
        dataset.append([image, directory])


Picking color scale for dataset.

In [ ]:
path = os.path.join(main_directory, 'paper')
item = os.listdir(path)
imageBGR = cv2.imread(os.path.join(path, item[0]))
imageRGB = cv2.cvtColor(imageBGR, cv2.COLOR_BGR2RGB)
imageGray = cv2.cvtColor(imageBGR, cv2.COLOR_BGR2GRAY)

plt.figure(figsize=(20,20))
plt.subplot(1, 3, 1)
plt.title("BGR")
plt.imshow(imageBGR)

plt.subplot(1, 3, 2)
plt.title("RGB")
plt.imshow(imageRGB)

plt.subplot(1, 3, 3)
plt.title("GRAY")
plt.imshow(imageGray)

plt.show()

In [ ]:
data, labels = zip(*dataset)
target = []
for label in labels:
    target.append(class_map[label])

In [ ]:
data = shuffle(data, random_state = 0)